# Common functions

So far, only contains what is expected to be a heavily reused load-and-parse method.

In [ ]:
public T[] LoadAndParse<T>(string path, Func<string, T> parse = null) {  
    parse = parse ?? (p => (T) Convert.ChangeType(p, typeof(T)));
    var rawInput = System.IO.File.ReadAllLines(path);
    return rawInput
    .Where(p=>!string.IsNullOrWhiteSpace(p))
    .Select(parse)
    .ToArray();
}

# --- Day 1: Not Quite Lisp ---


In [ ]:
{
    var input = System.IO.File.ReadAllText(@"day1_input.txt");
    int? firstNegative = null;
    int currentFloor = 0;
    for(int i = 0; i < input.Length; i++)   {
        if(input[i] == '(')
            currentFloor++;
        else
            currentFloor--;
        if(currentFloor < 0)
            firstNegative ??= (i+1);
    }
    Console.WriteLine($"Last floor: {currentFloor}");
    Console.WriteLine($"Santa first enters the basement on level {firstNegative}");
}

Last floor: 232
Santa first enters the basement on level 1783


# --- Day 2: I Was Told There Would Be No Math ---

In [ ]:
record Prism (int L, int W, int H) {
    //Both methods assume the coordinates are ordered l - w - h smallest to largest dimension.
    public int Area => 3*L*W + 2*W*H + 2*H*L;
    public int RibbonLength => 2*L + 2*W + L*W*H;
};

var input = LoadAndParse<Prism>(@"day2_input.txt", p => {
    var input = p.Split("x")
    .Select(p=>Int32.Parse(p))
    .OrderBy(p=>p)
    .ToArray();
    return new Prism(input[0], input[1], input[2]);
});
    
Console.WriteLine(input.Sum(p=>p.Area));
Console.WriteLine(input.Sum(p=>p.RibbonLength));

1606483
3842356


# --- Day 3: Perfectly Spherical Houses in a Vacuum ---

In [ ]:
var input = System.IO.File.ReadAllText(@"day3_input.txt");

int Visits(IEnumerable<IEnumerable<char>> paths) {
    System.Collections.Generic.HashSet<(int x, int y)> Visited = new();
    Visited.Add((0,0));
    
    foreach (var path in paths){
        (int x, int y) currentLocation = (0,0);
        foreach(char direction in path){
            switch (direction) {
                case '>': currentLocation = (currentLocation.x + 1, currentLocation.y); break;
                case '<': currentLocation = (currentLocation.x - 1, currentLocation.y); break;
                case '^': currentLocation = (currentLocation.x, currentLocation.y + 1); break;
                case 'v': currentLocation = (currentLocation.x, currentLocation.y - 1); break;
            }
            Visited.Add(currentLocation);
        }
    }
    return Visited.Count();   
}

Console.WriteLine(Visits(new[]{input}));

var inputTrails = input.Select((p, i) => (instruction: p, follower: i%2)).ToLookup(p=>p.follower, p=>p.instruction);
Console.WriteLine(Visits(inputTrails));

2592
2360


# --- Day 4: The Ideal Stocking Stuffer ---

In [ ]:
var input = System.IO.File.ReadAllText(@"day4_input.txt");
public int FindPrefixCollision(string target, int prefixZeroCount){
    using (var md5 = System.Security.Cryptography.MD5.Create()){
        int probe = 0;
        byte[] hash;
        
        while(true){
            //Note: Significant allocation overhead could probably be eliminated by in-place manipulating the byte-array.
            var bytes = System.Text.Encoding.ASCII.GetBytes($"{input}{probe}");
            hash = md5.ComputeHash(bytes);
            
            byte test = hash.Take(prefixZeroCount / 2).Aggregate((p, q) => (byte) (p | q));
            if(prefixZeroCount % 2 == 1)
                test |= (byte) (hash[prefixZeroCount/2] >> 4);

            if(test == 0){
                Console.WriteLine($"{prefixZeroCount} zero prefix collision found at probe {probe}. {input}{probe} resolves to MD5 {BitConverter.ToString(hash)}");
                return probe;
            }
            
            probe++;
        }
    }
    
}
FindPrefixCollision(input, 5);
FindPrefixCollision(input, 6);

5 zero prefix collision found at probe 282749. yzbqklnj282749 resolves to MD5 00-00-02-C6-55-DF-77-38-24-6E-88-F6-C1-C4-3E-B7
6 zero prefix collision found at probe 9962624. yzbqklnj9962624 resolves to MD5 00-00-00-4B-34-7B-F4-B3-98-B3-F6-2A-CE-7C-D3-01
